## Download the Data

In [1]:
from ucimlrepo import fetch_ucirepo 
  
# Downlaod the Dataset 
taiwanese_bankruptcy_prediction = fetch_ucirepo(id=572) 
  
# Get the features and target variables
X = taiwanese_bankruptcy_prediction.data.features 
y = taiwanese_bankruptcy_prediction.data.targets 
  


## Pre-process the data

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Split the dataset into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

# Preprocess the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Build the Model

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.metrics import Precision, Recall


model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy', Precision(), Recall()])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5964 - loss: 456494080.0000 - precision_5: 0.0438 - recall_5: 0.4420 - val_accuracy: 0.9707 - val_loss: 19350922.0000 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 2/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - accuracy: 0.9472 - loss: 19928768.0000 - precision_5: 0.0207 - recall_5: 0.0115 - val_accuracy: 0.9468 - val_loss: 11666447.0000 - val_precision_5: 0.0556 - val_recall_5: 0.0769
Epoch 3/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 671us/step - accuracy: 0.9334 - loss: 10989819.0000 - precision_5: 0.0460 - recall_5: 0.0445 - val_accuracy: 0.9643 - val_loss: 8206552.5000 - val_precision_5: 0.1176 - val_recall_5: 0.0769
Epoch 4/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 689us/step - accuracy: 0.9427 - loss: 6479104.0000 - precision_5: 0.1344 - recall_5: 0.1153 - val_accuracy: 0.9560 - val_loss: 6624834.5000 - val_precision_5: 0.0417 - val_recall_5: 0.0385
Epoch 5/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step -

## Evaluate the Model

In [16]:

# Evaluate the model
loss, accuracy, precision, recall = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}, Precision: {precision}, Recall: {recall}")

43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 525us/step - accuracy: 0.9111 - loss: 4204955.5000 - precision_5: 0.0583 - recall_5: 0.1083
Test loss: 3980375.75, Test accuracy: 0.9120234847068787, Precision: 0.0470588244497776, Recall: 0.09302325546741486
